In [1]:
from src.utils.spark_utils import SparkUtils 

# create a spark session 
spark = SparkUtils(app_name="nyc-taxi-data-ingestion").spark

# read the data from the parquet file 
df = spark.read.parquet("gold/2025/january/trip_01.parquet")

df.show()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/09 02:31:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/09 02:31:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/09 02:32:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------+------------+--------+--------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+------------------+-----------+-------------------+-------------------+------------+-----------+------------------+----------+------------+--------+------------+-------------------+------------------+------------------+--------------------+-------------+------------+-------------------+------------------+------------+----------------------+-----------------------+----------------+------------------+--------------------+--------------------+---------+---------------+------------------+------------------+
|route_id|DOLocationID|PULocationID|VendorID|tpep_pickup_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcha

In [4]:
## count the number of rows in dataframe 
df.count()





3250902

26/02/09 04:52:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 221714 ms exceeds timeout 120000 ms
26/02/09 04:52:44 WARN SparkContext: Killing executors is not supported by current scheduler.
26/02/09 04:52:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [3]:
df.show(5)

+--------+------------+------------+--------+--------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+------------------+-----------+-------------------+-------------------+------------+-----------+------------------+----------+------------+--------+------------+--------+--------+------------------+--------------------+-------------+------------+------------------+------------------+------------+----------------------+-----------------------+----------------+------------------+-------------------+----------+---------+---------------+------------------+------------------+
|route_id|DOLocationID|PULocationID|VendorID|tpep_pickup_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|Airport_fee|cbd_congestion_fe

In [3]:
from src.utils.spark_utils import SparkUtils 

# create a spark session 
spark = SparkUtils(app_name="nyc-taxi-data-ingestion").spark

# read the data from the parquet file 
df_train = spark.read.parquet("ml_transformed/train.parquet")

df_train.show(5)

+-------------------+--------------------+------------------+------------------+-----------+-------------------+-------------------+------------+-----------+------------------+----------+------------+--------+------------+--------+--------+------------------+------------------+-------------+------------+--------------------+--------------------+------------+----------------------+-----------------------+--------------------+---------+---------------+------------------+----------+----------+------------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+--------------------+--------------------+
|    passenger_count|       trip_distance|store_and_fwd_flag| trip_duration_min|pickup_hour|pickup_day_of_month|pickup_week_of_year|pickup_month|pickup_year|pickup_day_of_week|is_weekend|is_peak_hour|is_night|is_rush_hour|sin_hour|cos_hour|   sin_day_of_week|   cos_day_of_week|is_short_trip|is_long_trip|   log_trip_distance|  sqrt_trip

In [7]:
LABEL_COL = "trip_duration_min"

FEATURE_COLS = [
    "passenger_count",
    "trip_distance",
    "store_and_fwd_flag",
    "pickup_hour",
    "pickup_day_of_month",
    "pickup_week_of_year",
    "pickup_month",
    "pickup_year",
    "pickup_day_of_week",
    "is_weekend",
    "is_peak_hour",
    "is_night",
    "is_rush_hour",
    "sin_hour",
    "cos_hour",
    "sin_day_of_week",
    "cos_day_of_week",
    "is_short_trip",
    "is_long_trip",
    "log_trip_distance",
    "sqrt_trip_distance",
    "is_same_zone",
    "pickup_zone_trip_count",
    "dropoff_zone_trip_count",
    "route_trip_count",
    "has_tolls",
    "has_airport_fee",
    "has_congestion_fee",
    "VendorID_2",
    "VendorID_6",
    "RatecodeID_2",
    "RatecodeID_3",
    "RatecodeID_4",
    "RatecodeID_5",
    "RatecodeID_6",
    "payment_type_1",
    "payment_type_2",
    "payment_type_3",
    "payment_type_4",
    "DOLocationID_encoded",
    "PULocationID_encoded",
]


In [8]:
X = df_train.select(FEATURE_COLS)
y = df_train.select(LABEL_COL)

# sanity checks (do NOT skip)
assert LABEL_COL not in X.columns
assert X.count() == y.count()


In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=FEATURE_COLS,
    outputCol="features",
    handleInvalid="skip"
)

df_ml = assembler.transform(df_train).select(
    "features",
    df_train[LABEL_COL].alias("label")
)


In [10]:
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)


In [11]:
train_df.show(5)

+--------------------+------------------+
|            features|             label|
+--------------------+------------------+
|(41,[0,1,2,3,4,5,...|14.883333333333333|
|(41,[0,1,2,3,4,5,...|               2.5|
|(41,[0,1,2,3,4,5,...|2.4833333333333334|
|(41,[0,1,2,3,4,5,...|13.283333333333333|
|(41,[0,1,2,3,4,5,...|              4.25|
+--------------------+------------------+
only showing top 5 rows



In [12]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    regParam=0.1,
    elasticNetParam=0.0
)

lr_model = lr.fit(train_df)


26/02/10 19:20:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/10 19:20:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lr_model.transform(test_df)

rmse_eval = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

r2_eval = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

rmse = rmse_eval.evaluate(predictions)
r2 = r2_eval.evaluate(predictions)

print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


RMSE: 6.0541
R²:   0.7415


In [14]:
print(lr_model.coefficients)
print(lr_model.intercept)


[0.1719087083733641,1.8104686500059919,0.09177274075139016,-0.030566579792786246,0.004002422604904318,0.2056568953903814,-3.434961779053176,0.0,-0.5488886445216381,1.261881781628409,0.35367435386963564,1.0523846867216087,0.3536743538696281,-2.030928313362321,-3.5694830611519675,0.5290851962789009,-1.9690435773741066,-0.889567662979911,1.8475959862328277,4.769775566316091,3.0338882507549023,-0.5078535259924332,1.1496935989248631,1.1989728793106744,-1.1719188823118827,-2.5416381420874004,-3.5536667465691987,2.796199468462072,-0.21383431517251597,-3.3041191666914975,1.527922775416217,-1.197607326988876,-4.578314819548498,0.1355416830880566,20.408661894188427,-2.8835944897195462,-2.6294143839972595,-2.330921925680239,-2.301763066753251,0.1715916042880345,0.1411409043825525]
23.245349531518425


26/02/11 04:58:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1601425 ms exceeds timeout 120000 ms
26/02/11 04:58:54 WARN SparkContext: Killing executors is not supported by current scheduler.
26/02/11 04:58:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.